## Ch05 Factorization Machines

추천시스템의 최종 목표는 각 사용자의 각 아이템에 대한 선호도(구매 확률)를 예측하는 것이다.  
Factorization Machines(FM)은 사용자와 아이템의 다양한 특성을 모델화함으로써 예측의 성능을 높이려는 방법이다.

MF 모델은 사용자의 취향과 아이템의 특성으로 각 사용자의 각 아이템의 선호도를 예측하는 방식이다.  
그런데 사용자의 취향과 아이템의 특성뿐 아니라 예측에 도움을 줄 수 있는 다른 변수가 존재할 수 있다.  
성별, 지역 등 인구통계 변수나 구매 시점 등의 변수를 추가하면 더 정확해지는데 이러한 변수를 종합해서 요인화 해주는 방법이 FM이다.

### 5.1 FM의 표준식

FM의 기본 아이디어는 모든 변수와 그 변수들 간의 상호작용을 고려하여 평점을 예측하는 것이다.  
영화 평점 예측을 예를 들면 모든 변수가 영화의 평점에 영향을 학습하고 이를 바탕으로 평점을 예측하는 모델이다.  

<img src = 'https://velog.velcdn.com/images/amzyoungchae/post/16c06119-7446-42db-a516-f7519aa86a1e/image.png' width = 500 height = 300>

$\hat y$ : 예측값  
$x$ : input variable(입력 변수)  
$w_0$ : global bias(전체 편향, 전체 평균)  
$w_i$ : bias(입력 변수 $x_i$의 편향)  
$v_i$ : latent matrix(잠재 요인 행렬) $v$에서 변수 $x_i$의 특성값  
$n$ : n of input variable(입력 변수의 수)  
$k$ : n of latent factors(잠재요인의 수)

FM은 MF에 다른 변수를 추가할 수 있도록 좀 더 일반화한 모델이다.

### 5.2 FM 식의 변형

<img src='https://velog.velcdn.com/images/amzyoungchae/post/0ca848bd-8cb8-4dbd-b21b-75df2b16a897/image.png'>

FM 의 표준식은 계산의 효율성이 좋지 않기 때문에 FM을 실제로 적용할 때에는 변형된 식을 사용한다.

### 5.3 FM의 학습

FM 학습은 다른 기계학습 방식과 동일하게 진행되는데 아래와 같은 절차를 따른다.
1. 비용함수를 설정한다. (예를 들면 RMSE)
2. $w_0, w, v$를 초기화한다.
3. 주어진 $w_0, w, v$에 따라 비용함수를 계산한다.
4. $w,v$ 의 update rule 에 따라 $w,v$ 를 업데이트한다.
5. 비용함수가 더 이상 개선되지 않을 떄까지 3,4 단계를 반복한다.

### 5.4 FM의 데이터 변형

### 5.5 Python으로 FM의 구현

다른 변수를 사용하는 예는 뒤에서 설명하고 사용자 id와 아이템 id 만 사용하는 FM의 구현 예를 살펴보자.  
데이터를 읽어서 sparse matrix 형태로 변형하는 코드이다.

In [2]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

# 데이터 읽기
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1') 

# User encoding
user_dict = {}
for i in set(ratings['user_id']):
    user_dict[i] = len(user_dict)
n_user = len(user_dict)

# Item encoding
item_dict = {}
start_point = n_user
for i in set(ratings['movie_id']):
    item_dict[i] = start_point + len(item_dict)
n_item = len(item_dict)
start_point += n_item
num_x = start_point # Total number of x
ratings = shuffle(ratings, random_state=42)

In [3]:
# Generate X data
data = []
y = []
w0 = np.mean(ratings['rating'])
for i in range(len(ratings)):
    case = ratings.iloc[i]
    x_index = []
    x_value = []
    x_index.append(user_dict[case['user_id']]) # User id encoding
    x_value.append(1)
    x_index.append(item_dict[case['movie_id']]) # Movie id encoding
    x_value.append(1)
    data.append([x_index, x_value])
    y.append(case['rating'] - w0)
    if (i % 10000) == 0:
        print('Encoding ', i, ' cases...')

Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...
Encoding  30000  cases...
Encoding  40000  cases...
Encoding  50000  cases...
Encoding  60000  cases...
Encoding  70000  cases...
Encoding  80000  cases...
Encoding  90000  cases...


다음은 FM을 구현한 클래스이다.

In [4]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

class FM():
    def __init__(self, N, K, data, y, alpha, beta, train_ratio=0.75, iterations=100, tolerance=0.005, l2_reg=True, verbose=True):
        self.K = K          # Number of latent factors
        self.N = N          # Number of x (variables)
        self.n_cases = len(data)            # N of observations
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.l2_reg = l2_reg
        self.tolerance = tolerance
        self.verbose = verbose
        # w 초기화
        self.w = np.random.normal(scale=1./self.N, size=(self.N))
        # v 초기화
        self.v = np.random.normal(scale=1./self.K, size=(self.N, self.K))
        # Train/Test 분리
        cutoff = int(train_ratio * len(data))
        self.train_x = data[:cutoff]
        self.test_x = data[cutoff:]
        self.train_y = y[:cutoff]
        self.test_y = y[cutoff:]

    def test(self):                                     # Training 하면서 RMSE 계산 
        # SGD를 iterations 숫자만큼 수행
        best_RMSE = 10000
        best_iteration = 0
        training_process = []
        for i in range(self.iterations):
            rmse1 = self.sgd(self.train_x, self.train_y)        # SGD & Train RMSE 계산
            rmse2 = self.test_rmse(self.test_x, self.test_y)    # Test RMSE 계산     
            training_process.append((i, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1, rmse1, rmse2))
            if best_RMSE > rmse2:                       # New best record
                best_RMSE = rmse2
                best_iteration = i
            elif (rmse2 - best_RMSE) > self.tolerance:  # RMSE is increasing over tolerance
                break
        print(best_iteration, best_RMSE)
        return training_process
        
    # w, v 업데이트를 위한 Stochastic gradient descent 
    def sgd(self, x_data, y_data):
        y_pred = []
        for data, y in zip(x_data, y_data):
            x_idx = data[0]
            x_0 = np.array(data[1])     # xi axis=0 [1, 2, 3]
            x_1 = x_0.reshape(-1, 1)    # xi axis=1 [[1], [2], [3]]
    
            # biases
            bias_score = np.sum(self.w[x_idx] * x_0)
    
            # score 계산
            vx = self.v[x_idx] * (x_1)          # v matrix * x
            sum_vx = np.sum(vx, axis=0)         # sigma(vx)
            sum_vx_2 = np.sum(vx * vx, axis=0)  # ( v matrix * x )의 제곱
            latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

            # 예측값 계산
            y_hat = bias_score + latent_score
            y_pred.append(y_hat)
            error = y - y_hat
            # w, v 업데이트
            if self.l2_reg:     # regularization이 있는 경우
                self.w[x_idx] += error * self.alpha * (x_0 - self.beta * self.w[x_idx])
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1) - self.beta * self.v[x_idx])
            else:               # regularization이 없는 경우
                self.w[x_idx] += error * self.alpha * x_0
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1))
        return RMSE(y_data, y_pred)

    def test_rmse(self, x_data, y_data):
        y_pred = []
        for data , y in zip(x_data, y_data):
            y_hat = self.predict(data[0], data[1])
            y_pred.append(y_hat)
        return RMSE(y_data, y_pred)

    def predict(self, idx, x):
        x_0 = np.array(x)
        x_1 = x_0.reshape(-1, 1)

        # biases
        bias_score = np.sum(self.w[idx] * x_0)

        # score 계산
        vx = self.v[idx] * (x_1)
        sum_vx = np.sum(vx, axis=0)
        sum_vx_2 = np.sum(vx * vx, axis=0)
        latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

        # 예측값 계산
        y_hat = bias_score + latent_score
        return y_hat

In [5]:
K = 350
fm1 = FM(num_x, K, data, y, alpha = 0.0014, beta = 0.075, train_ratio= 0.75, iterations=300, tolerance=0.0005, l2_reg=True, verbose=True)
result = fm1.test()

Iteration: 10 ; Train RMSE = 0.956651 ; Test RMSE = 0.972614
Iteration: 20 ; Train RMSE = 0.935132 ; Test RMSE = 0.956315
Iteration: 30 ; Train RMSE = 0.926349 ; Test RMSE = 0.949946
Iteration: 40 ; Train RMSE = 0.921554 ; Test RMSE = 0.946727
Iteration: 50 ; Train RMSE = 0.918415 ; Test RMSE = 0.944877
Iteration: 60 ; Train RMSE = 0.915865 ; Test RMSE = 0.943649
Iteration: 70 ; Train RMSE = 0.913073 ; Test RMSE = 0.942575
Iteration: 80 ; Train RMSE = 0.908936 ; Test RMSE = 0.941146
Iteration: 90 ; Train RMSE = 0.901689 ; Test RMSE = 0.938625
Iteration: 100 ; Train RMSE = 0.889230 ; Test RMSE = 0.934301
Iteration: 110 ; Train RMSE = 0.871065 ; Test RMSE = 0.928619
Iteration: 120 ; Train RMSE = 0.848864 ; Test RMSE = 0.923226
Iteration: 130 ; Train RMSE = 0.823247 ; Test RMSE = 0.918859
Iteration: 140 ; Train RMSE = 0.793320 ; Test RMSE = 0.915355
Iteration: 150 ; Train RMSE = 0.758559 ; Test RMSE = 0.912760
Iteration: 160 ; Train RMSE = 0.719489 ; Test RMSE = 0.911336
Iteration: 170 ; 

이번에는 사용자, 아이템 외의 추가 데이터까지 사용하는 경우를 살펴보기로 한다.  
MovieLens 에 있는 데이터 중에서 사용자의 occupation, gender, age, genre를 사용한다.

In [6]:
# 데이터 읽기
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./data/u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('./data/u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_cols, encoding='latin-1') 

In [7]:
# User encoding
user_dict = {}
for i in set(users['user_id']):
    user_dict[i] = len(user_dict)
n_user = len(user_dict)

# Item encoding
item_dict = {}
start_point = n_user
for i in set(movies['movie_id']):
    item_dict[i] = start_point + len(item_dict)
n_item = len(item_dict)
start_point += n_item

# Occupation encoding
occ_dict = {}
for i in set(users['occupation']):
    occ_dict[i] = start_point + len(occ_dict)
n_occ = len(occ_dict)
start_point += n_occ

# Gender encoding
gender_dict = {}
for i in set(users['sex']):
    gender_dict[i] = start_point + len(gender_dict)
n_gender = len(gender_dict)
start_point += n_gender

# Genre encoding
genre_dict = {}
genre = ['unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
for i in genre:
    genre_dict[i] = start_point + len(genre_dict)
n_genre = len(genre_dict)
start_point += n_genre
age_index = start_point
start_point += 1
num_x = start_point # Total number of x

In [8]:
# Merge data
movies = movies.drop(['title', 'release date', 'video release date', 'IMDB URL'], axis=1)
users = users.drop(['zip_code'], axis=1)
ratings = ratings.drop(['timestamp'], axis=1)

x = pd.merge(ratings, movies, how = 'outer', on = 'movie_id')
x = pd.merge(x, users, how = 'outer', on = 'user_id')
x = shuffle(x, random_state = 1)

In [9]:
# Generate X data
data = []
y = []
age_mean = np.mean(x['age'])
age_std = np.std(x['age'])
w0 = np.mean(x['rating'])
for i in range(len(x)):
    case = x.iloc[i]
    x_index = []
    x_value = []
    x_index.append(user_dict[case['user_id']]) # User id encoding
    x_value.append(1)
    x_index.append(item_dict[case['movie_id']]) # Movie id encoding
    x_value.append(1)
    x_index.append(occ_dict[case['occupation']]) # Occupation id encoding
    x_value.append(1)
    x_index.append(gender_dict[case['sex']]) # Gender id encoding
    x_value.append(1)
    for j in genre:
        if case[j] == 1: # 해당 장르가 1
            x_index.append(genre_dict[j])
            x_value.append(1)
    x_index.append(age_index)
    x_value.append((case['age'] - age_mean) / age_std)
    data.append([x_index, x_value])
    y.append(case['rating'] - w0)
    if (i % 10000) == 0:
        print('Encoding ', i, ' cases...')

Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...
Encoding  30000  cases...
Encoding  40000  cases...
Encoding  50000  cases...
Encoding  60000  cases...
Encoding  70000  cases...
Encoding  80000  cases...
Encoding  90000  cases...


In [10]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

class FM():
    def __init__(self, N, K, data, y, alpha, beta, train_ratio=0.75, iterations=100, tolerance=0.005, l2_reg=True, verbose=True):
        self.K = K          # Number of latent factors
        self.N = N          # Number of x (variables)
        self.n_cases = len(data)            # N of observations
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.l2_reg = l2_reg
        self.tolerance = tolerance
        self.verbose = verbose
        # w 초기화
        self.w = np.random.normal(scale=1./self.N, size=(self.N))
        # v 초기화
        self.v = np.random.normal(scale=1./self.K, size=(self.N, self.K))
        # Train/Test 분리
        cutoff = int(train_ratio * len(data))
        self.train_x = data[:cutoff]
        self.test_x = data[cutoff:]
        self.train_y = y[:cutoff]
        self.test_y = y[cutoff:]

    def test(self):                                     # Training 하면서 RMSE 계산 
        # SGD를 iterations 숫자만큼 수행
        best_RMSE = 10000
        best_iteration = 0
        training_process = []
        for i in range(self.iterations):
            rmse1 = self.sgd(self.train_x, self.train_y)        # SGD & Train RMSE 계산
            rmse2 = self.test_rmse(self.test_x, self.test_y)    # Test RMSE 계산     
            training_process.append((i, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1, rmse1, rmse2))
            if best_RMSE > rmse2:                       # New best record
                best_RMSE = rmse2
                best_iteration = i
            elif (rmse2 - best_RMSE) > self.tolerance:  # RMSE is increasing over tolerance
                break
        print(best_iteration, best_RMSE)
        return training_process
    
    # w, v 업데이트를 위한 Stochastic gradient descent 
    def sgd(self, x_data, y_data):
        y_pred = []
        for data, y in zip(x_data, y_data):
            x_idx = data[0]
            x_0 = np.array(data[1])     # xi axis=0 [1, 2, 3]
            x_1 = x_0.reshape(-1, 1)    # xi axis=1 [[1], [2], [3]]
    
            # biases
            bias_score = np.sum(self.w[x_idx] * x_0)
            
            # score 계산
            vx = self.v[x_idx] * (x_1)          # v matrix * x
            sum_vx = np.sum(vx, axis=0)         # sigma(vx)
            sum_vx_2 = np.sum(vx * vx, axis=0)  # ( v matrix * x )의 제곱
            latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

            # 예측값 계산
            y_hat = bias_score + latent_score
            y_pred.append(y_hat)
            error = y - y_hat
            # w, v 업데이트
            if self.l2_reg:     # regularization이 있는 경우
                self.w[x_idx] += error * self.alpha * (x_0 - self.beta * self.w[x_idx])
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1) - self.beta * self.v[x_idx])
            else:               # regularization이 없는 경우
                self.w[x_idx] += error * self.alpha * x_0
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1))
        return RMSE(y_data, y_pred)
    
    def test_rmse(self, x_data, y_data):
        y_pred = []
        for data , y in zip(x_data, y_data):
            y_hat = self.predict(data[0], data[1])
            y_pred.append(y_hat)
        return RMSE(y_data, y_pred)

    def predict(self, idx, x):
        x_0 = np.array(x)
        x_1 = x_0.reshape(-1, 1)

        # biases
        bias_score = np.sum(self.w[idx] * x_0)

        # score 계산
        vx = self.v[idx] * (x_1)
        sum_vx = np.sum(vx, axis=0)
        sum_vx_2 = np.sum(vx * vx, axis=0)
        latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

        # 예측값 계산
        y_hat = bias_score + latent_score
        return y_hat

In [11]:
K = 200
fm1 = FM(num_x, K, data, y, alpha=0.00005, beta=0.002, train_ratio=0.75, iterations=300, tolerance=0.0001, l2_reg=True, verbose=True)
result = fm1.test()

Iteration: 10 ; Train RMSE = 1.077456 ; Test RMSE = 1.080493
Iteration: 20 ; Train RMSE = 1.053051 ; Test RMSE = 1.057796
Iteration: 30 ; Train RMSE = 1.026664 ; Test RMSE = 1.033892
Iteration: 40 ; Train RMSE = 0.999221 ; Test RMSE = 1.010252
Iteration: 50 ; Train RMSE = 0.974847 ; Test RMSE = 0.990357
Iteration: 60 ; Train RMSE = 0.955982 ; Test RMSE = 0.975819
Iteration: 70 ; Train RMSE = 0.942098 ; Test RMSE = 0.965743
Iteration: 80 ; Train RMSE = 0.931706 ; Test RMSE = 0.958702
Iteration: 90 ; Train RMSE = 0.923697 ; Test RMSE = 0.953708
Iteration: 100 ; Train RMSE = 0.917357 ; Test RMSE = 0.950123
Iteration: 110 ; Train RMSE = 0.912208 ; Test RMSE = 0.947508
Iteration: 120 ; Train RMSE = 0.907910 ; Test RMSE = 0.945558
Iteration: 130 ; Train RMSE = 0.904221 ; Test RMSE = 0.944062
Iteration: 140 ; Train RMSE = 0.900972 ; Test RMSE = 0.942881
Iteration: 150 ; Train RMSE = 0.898041 ; Test RMSE = 0.941923
Iteration: 160 ; Train RMSE = 0.895344 ; Test RMSE = 0.941126
Iteration: 170 ; 